### Intrusion detection using SURICATA



In [ ]:
# on the gateway: creating an intrusion prevention system
mkdir /etc/systemd/system/suricata.service.d

-D option: stars suricata in daemon mode: executes in the background  
-q: suricata has to handle itself the packets put aside by the firewall through the target NFQUEUE
-c etc... : specifies the config parameters and rules of the YAML config file  
--pidfile /var.... indicates where to save the PID file of suricata

In [ ]:
# in  /etc/systemd/system/suricata.service.d/suricata.conf
[Service]
ExecStart =
ExecStart=/usr/bin/suricata -D -q 0 -c /etc/suricata/suricata.yaml --pidfile /var/run/suricata.pid

You need to run `systemctl daemon-reload` after modifying service unit files to inform systemd to reload its configuration and recognize any changes made. This ensures that the latest configurations are applied, preventing inconsistencies and ensuring that services operate with the updated settings.

In [ ]:
# do not forget to reload
systemctl daemon-reload

In [ ]:
# edit the /etc/init.d/ma-config.sh file to specify the trafic to send to suricata

iptables -A FORWARD -j NFQUEUE

In [ ]:
# creating a file to store the rules for suricata
toucj /etc/suricata/rules/myrules.rules

# indicates to suricata the rules file to load
# in /etc/suricata/suricata.yaml
[ . . . ]

rule-files:
  - myrules.rules

[ . . . ]

In [ ]:
# adding our first rule to our rules file
# sends an alert if an ICMP packet go through our gateway
alert icmp any any -> any any ("msg: SURICATA my ICMP alert"; sid:22400321)

In [ ]:
# do not forget to restart service
service suricata restart

In [ ]:
# check the log messages file
cat /var/log/suricata/fast.log

In [ ]:
# adding a rule: alerting if one fails to enter valid password

# initialize counter
alert tcp-pkt any any <> any 23 (msg:"Init Failing logins counter";\
  content:"Login incorrect";flow:established,from_server;\
  flowint: username, notset;\
  flowint:username, =, 1; noalert; sid:1;)

# increase counter
alert tcp-pkt any any <> any 23 (msg:"Inc Failed Logins";\
  content:"Login incorrect";flow:established,from_server;\
  flowint: username, isset;\
  flowint:username, +, 1; noalert; sid:2;)

# send an alert if counter is superior to 2
alert tcp-pkt any any <> any 23 (msg:"More than 3 Failed Logins!";\
  content:"Login incorrect"; flow:established,from_server; \
  flowint: username, isset;\
  flowint:username, >, 2; sid:3;)

To check if it worked, one must try to connect and fail three times and then check the logs.
Do not forget to restart services

In [ ]:
# connecting using telnet
telnet 10.0.0.2 23 # telnet port

#### Advanced feature of suricata: using a lua script

In [ ]:
# enable the TLS certificate store in /etc/suricata/suricata.yaml
- tls-store:
  enabled: yes
  certs-log-dir: certs

In [1]:
# add a rule
alert tls any any -> any any (msg:"SURICATA TLS Self Signed Certificate";
                 flow:established; luajit script.lua;
                 tls.store; classtype:protocol-command-decode; sid:99666321; rev:1;)

In [ ]:
# add the lua script to /etc/suricata/rules

In [ ]:
# listening using gnu:
gnutls-serv –priority SECURE256:-VERS-TLS-ALL:+VERS-TLS1.2 –x509certfile=/mnt/host/cert.pem –x509keyfile=/mnt/host/key.pem -p 5555